In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.api import add_constant
from statsmodels.sandbox.regression.gmm import IV2SLS

%store -r full
%store -r solution

%store -r novelty_b3_p1
%store -r novelty_b3_p2
%store -r novelty_b3_p3
%store -r novelty_b3_p4

%store -r novelty_b2_p1
%store -r novelty_b2_p2
%store -r novelty_b2_p3
%store -r novelty_b2_p4

%store -r overall_loc_scores

In [12]:
df_novelty = pd.read_csv('scores_novelty.csv')
df_novelty

,Unnamed: 0,user1,user2,user1_phase,user2_phase,similarity,stage,group,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,count,improve-ur,improve-vis,improve-total,improve-novel,improve-tech
0,0,juanescai,fffeelipe,1,2,0.000000,1,0,1.0,0.0,5.0,3.0,1.0,40.0,8.33,73.33,0.00,25.0
1,1,annipi,jdsolanos,1,2,0.000000,1,0,1.0,1.0,5.0,2.0,0.0,0.0,0.00,-50.00,0.00,-50.0
2,2,jdsolanos,annipi,1,2,0.000000,1,0,1.0,0.0,4.0,2.0,0.0,0.0,0.00,12.50,0.00,12.5
3,3,jdsolanos,jcneivaa,1,2,0.000000,1,0,0.0,0.0,4.0,2.0,0.0,0.0,0.00,12.50,0.00,12.5
4,4,katherinsb,juanescai,1,2,0.000000,1,0,1.0,2.0,5.0,2.0,1.0,0.0,0.00,-50.00,-37.50,-12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,614,lacardenasv,jorge52an,4,5,0.931937,4,3,4.0,4.0,5.0,2.0,12.0,40.0,16.67,66.67,-15.00,25.0
615,615,NicolasPrr,capinzor,4,5,0.964725,4,3,0.0,0.0,2.0,2.0,6.0,0.0,0.00,8.33,-4.17,12.5
616,616,juligarji,dduartec,4,5,0.969512,4,3,1.0,5.0,4.0,1.0,1.0,80.0,83.33,258.33,95.00,0.0
617,617,capinzor,juligarji,4,5,0.970410,4,3,4.0,4.0,5.0,3.0,2.0,0.0,4.17,-3.33,5.00,-12.5


In [19]:
df_novelty_overall_23 = df_novelty[(df_novelty['group'] == 2) | (df_novelty['group'] == 3)]
df_novelty_overall_23['ln_novelty'] = np.log(df_novelty_overall_23['similarity'] + 1)
reg_expr = 'ln_novelty ~ group'
olsr_model = smf.ols(formula=reg_expr, data=df_novelty_overall_23)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:             ln_novelty   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                   0.01806
Date:                Fri, 25 Nov 2022   Prob (F-statistic):              0.893
Time:                        22:09:02   Log-Likelihood:                 70.904
No. Observations:                 303   AIC:                            -137.8
Df Residuals:                     301   BIC:                            -130.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4891      0.057      8.527      0.0

/var/folders/nr/2yp640v56_x34zrd2n5jvfqc0000gn/T/ipykernel_3342/2585275182.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novelty_overall_23['ln_novelty'] = np.log(df_novelty_overall_23['similarity'] + 1)


In [21]:
df_novelty_overall_23_stage4 = df_novelty_overall_23[df_novelty_overall_23['stage'] == 4]
reg_expr = 'ln_novelty ~ group'
olsr_model = smf.ols(formula=reg_expr, data=df_novelty_overall_23_stage4)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:             ln_novelty   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                   0.07571
Date:                Fri, 25 Nov 2022   Prob (F-statistic):              0.784
Time:                        22:13:55   Log-Likelihood:                 26.331
No. Observations:                  81   AIC:                            -48.66
Df Residuals:                      79   BIC:                            -43.87
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4913      0.101      4.887      0.0

In [15]:
df_novelty_overall_23 = df_novelty_overall_23[(df_novelty_overall_23['Group'] == 2) | (overall_loc_scores['Group'] == 3)]
df_novelty_overall_23

/var/folders/nr/2yp640v56_x34zrd2n5jvfqc0000gn/T/ipykernel_3342/4107926483.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_novelty_overall_23 = df_novelty_overall_23[(df_novelty_overall_23['Group'] == 2) | (overall_loc_scores['Group'] == 3)]


,Student,Group,user-requirement-phase5,infovis-phase5,novelty-phase5,total-phase5,tech-phase5,tech-phase4,user-requirement-phase4,infovis-phase4,...,add-loc-25,delete-loc-25,change-files-25,add-loc-sum,improve-effort-12,improve-effort-23,improve-effort-34,improve-effort-45,improve-effort-15,improve-effort-25
80,jusherrerama,2.0,20.0,25.00,0.0,120.00,75.0,62.5,20.0,25.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,bdvegat,2.0,20.0,25.00,0.0,107.50,62.5,50.0,20.0,25.00,...,3.0,4.0,2.0,25.0,0.0,0.0,0.0,0.0,0.0,1.0
82,Phoenixest,2.0,0.0,25.00,25.0,75.00,25.0,25.0,20.0,25.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83,HashNick,2.0,40.0,50.00,25.0,152.50,37.5,37.5,40.0,54.17,...,7.0,10.0,2.0,21.0,0.5,0.0,6.5,0.0,0.0,2.0
84,ccvacad,2.0,80.0,66.67,80.0,314.17,87.5,0.0,0.0,0.00,...,1.0,1.0,1.0,24.0,-17.5,0.0,-16.0,-9.5,-17.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,NicolasPrr,3.0,100.0,83.33,90.0,360.83,87.5,62.5,0.0,33.33,...,41.0,49.0,5.0,108.0,0.0,0.0,-36.5,0.0,16.5,16.5
159,jumcorredorro,3.0,100.0,62.50,75.0,325.00,87.5,87.5,80.0,58.33,...,29.0,35.0,1.0,32.0,0.0,12.0,1.0,1.0,11.5,11.5
160,feartheGru,3.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,6.0,234.0,2.0,6.0,-33.5,0.0,0.0,0.0,0.0,-111.0
161,Danielsv9207,3.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
novelty['ln_novelty'] = np.log(novelty['similarity'] + 1)

In [2]:
overall_23 = overall_loc_scores[(overall_loc_scores['Group'] == 2) | (overall_loc_scores['Group'] == 3)]
overall_23

,Student,Group,user-requirement-phase5,infovis-phase5,novelty-phase5,total-phase5,tech-phase5,tech-phase4,user-requirement-phase4,infovis-phase4,...,add-loc-25,delete-loc-25,change-files-25,add-loc-sum,improve-effort-12,improve-effort-23,improve-effort-34,improve-effort-45,improve-effort-15,improve-effort-25
80,jusherrerama,2.0,20.0,25.00,0.0,120.00,75.0,62.5,20.0,25.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,bdvegat,2.0,20.0,25.00,0.0,107.50,62.5,50.0,20.0,25.00,...,3.0,4.0,2.0,25.0,0.0,0.0,0.0,0.0,0.0,1.0
82,Phoenixest,2.0,0.0,25.00,25.0,75.00,25.0,25.0,20.0,25.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83,HashNick,2.0,40.0,50.00,25.0,152.50,37.5,37.5,40.0,54.17,...,7.0,10.0,2.0,21.0,0.5,0.0,6.5,0.0,0.0,2.0
84,ccvacad,2.0,80.0,66.67,80.0,314.17,87.5,0.0,0.0,0.00,...,1.0,1.0,1.0,24.0,-17.5,0.0,-16.0,-9.5,-17.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,NicolasPrr,3.0,100.0,83.33,90.0,360.83,87.5,62.5,0.0,33.33,...,41.0,49.0,5.0,108.0,0.0,0.0,-36.5,0.0,16.5,16.5
159,jumcorredorro,3.0,100.0,62.50,75.0,325.00,87.5,87.5,80.0,58.33,...,29.0,35.0,1.0,32.0,0.0,12.0,1.0,1.0,11.5,11.5
160,feartheGru,3.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,6.0,234.0,2.0,6.0,-33.5,0.0,0.0,0.0,0.0,-111.0
161,Danielsv9207,3.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
novelty = pd.concat([novelty_b3_p1, novelty_b3_p2, novelty_b3_p3, novelty_b3_p4, 
                    novelty_b2_p1, novelty_b2_p2, novelty_b2_p3, novelty_b2_p4], ignore_index=True)
novelty = novelty.fillna(0)
novelty

,Student,group,abs_perform_diff_best,phase,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,user2,novelty
0,lemartinp,3,270.84,1,0.0,0.0,4.0,1.0,0.146341,lemartinp,0.708333
1,juligarji,3,167.50,1,4.0,4.0,5.0,3.0,0.057325,juligarji,0.942675
2,nfmorenog,3,225.00,1,0.0,2.0,3.0,1.0,0.190476,nfmorenog,0.745098
3,AFelipeGA,3,291.67,1,4.0,3.0,5.0,2.0,0.003841,AFelipeGA,0.985581
4,mballeng91,3,329.17,1,2.0,6.0,6.0,2.0,0.004854,mballeng91,0.968912
...,...,...,...,...,...,...,...,...,...,...,...
327,xdanielsb,2,0.00,1,3.0,3.0,3.0,2.0,0.000000,xdanielsb,0.949627
328,CSebasGomez,2,260.00,1,3.0,3.0,5.0,2.0,0.000000,CSebasGomez,0.000000
329,jhcardenasa,2,260.00,1,4.0,4.0,5.0,3.0,0.000000,jhcardenasa,0.000000
330,tres-iqus,2,260.00,1,1.0,1.0,4.0,0.0,0.000000,0,0.000000


In [4]:
novelty.to_csv("novelty.csv")

In [5]:
commits = pd.read_csv("commits.csv")
commits

,Unnamed: 0.2,index,Student,Group,user-requirement-phase5,infovis-phase5,novelty-phase5,total-phase5,tech-phase5,tech-phase4,...,files,additions,deletions,email_name,email_y,hack_id,dates,dates.1,phase,count
0,0,0,fffeelipe,0.0,20.0,29.17,50.0,199.17,100.0,100.0,...,6.0,168.0,78.0,lfepiar,lfepiar@unal.edu.co,0.0,23/04/2017,23/04/2017,5.0,1
1,1,3,fffeelipe,0.0,20.0,29.17,50.0,199.17,100.0,100.0,...,5.0,281.0,295.0,lfepiar,lfepiar@unal.edu.co,0.0,13/04/2017,13/04/2017,4.0,3
2,2,4,fffeelipe,0.0,20.0,29.17,50.0,199.17,100.0,100.0,...,6.0,324.0,80.0,lfepiar,lfepiar@unal.edu.co,0.0,11/04/2017,11/04/2017,3.0,1
3,3,5,fffeelipe,0.0,20.0,29.17,50.0,199.17,100.0,100.0,...,5.0,108.0,1339.0,lfepiar,lfepiar@unal.edu.co,0.0,05/04/2017,05/04/2017,2.0,1
4,4,7,fffeelipe,0.0,20.0,29.17,50.0,199.17,100.0,100.0,...,5.0,382.0,2.0,lfepiar,lfepiar@unal.edu.co,0.0,30/03/2017,30/03/2017,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,535,1508,jumcorredorro,3.0,100.0,62.50,75.0,325.00,87.5,87.5,...,5.0,578.0,34.0,jumcorredorro,jumcorredorro@unal.edu.co,3.0,23/04/2017,23/04/2017,5.0,1
536,536,1509,jumcorredorro,3.0,100.0,62.50,75.0,325.00,87.5,87.5,...,3.0,266.0,12.0,jumcorredorro,jumcorredorro@unal.edu.co,3.0,17/04/2017,17/04/2017,4.0,1
537,537,1512,jumcorredorro,3.0,100.0,62.50,75.0,325.00,87.5,87.5,...,15.0,469.0,202.0,jumcorredorro,jumcorredorro@unal.edu.co,3.0,11/04/2017,11/04/2017,3.0,3
538,538,1514,jumcorredorro,3.0,100.0,62.50,75.0,325.00,87.5,87.5,...,10.0,316.0,32.0,jumcorredorro,jumcorredorro@unal.edu.co,3.0,02/04/2017,02/04/2017,2.0,2


In [6]:
full

,Unnamed: 0,Student,group,abs_perform_diff_best,phase,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity
0,0,lemartinp,3,270.84,1,0.0,0.0,4.0,1.0,0.146341,0.136576
1,1,juligarji,3,167.50,1,4.0,4.0,5.0,3.0,0.057325,0.055742
2,2,nfmorenog,3,225.00,1,0.0,2.0,3.0,1.0,0.190476,0.174353
3,3,AFelipeGA,3,291.67,1,4.0,3.0,5.0,2.0,0.090909,0.087011
4,4,mballeng91,3,329.17,1,2.0,6.0,6.0,2.0,0.004854,0.004843
...,...,...,...,...,...,...,...,...,...,...,...
159,159,NicolasPrr,3,237.50,4,3.0,4.0,3.0,2.0,0.009360,0.009316
160,160,jumcorredorro,3,53.33,4,1.0,1.0,5.0,2.0,0.013294,0.013206
161,161,feartheGru,3,358.33,4,2.0,2.0,5.0,2.0,0.000000,0.000000
162,162,Danielsv9207,3,358.33,4,1.0,1.0,1.0,3.0,0.000000,0.000000


In [7]:
full_phase4 = full[full['phase'] == 4]
full_phase4['ln_similarity'].mean()

0.006938294589454749

In [8]:
full_phase3 = full[full['phase'] == 3]
full_phase3['ln_similarity'].mean()

0.06400689379074434

In [9]:
solution

,Unnamed: 0,Student,group,abs_perform_diff_best,phase,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity
1,1,bdvegat,2,260.00,1,1.0,2.0,4.0,1.0,0.054187,0.052770
2,2,Phoenixest,2,185.83,1,0.0,1.0,3.0,1.0,0.017964,0.017805
3,3,HashNick,2,260.00,1,1.0,3.0,5.0,3.0,0.023697,0.023420
4,4,ccvacad,2,260.00,1,1.0,0.0,4.0,2.0,0.000000,0.000000
5,5,joaortizro,2,152.50,1,0.0,2.0,2.0,1.0,0.041420,0.040585
...,...,...,...,...,...,...,...,...,...,...,...
162,162,lsfinite,2,358.33,4,1.0,1.0,6.0,3.0,0.008392,0.008357
163,163,xdanielsb,2,283.33,4,3.0,3.0,3.0,2.0,0.040128,0.039344
164,164,CSebasGomez,2,358.33,4,3.0,3.0,5.0,2.0,0.000000,0.000000
165,165,jhcardenasa,2,358.33,4,4.0,4.0,5.0,3.0,0.051020,0.049762


In [10]:
solution_phase4 = solution[solution['phase'] == 4]
solution_phase4['ln_similarity'].mean()

0.04194164073788817

In [11]:
solution_phase3 = solution[solution['phase'] == 3]
solution_phase3['ln_similarity'].mean()

0.05522053950410895

In [12]:
df = solution.append(full, ignore_index=True)
df

/var/folders/gf/pvtbdsj96cz1tv7vmn1gq7j40000gn/T/ipykernel_4277/152460129.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = solution.append(full, ignore_index=True)


,Unnamed: 0,Student,group,abs_perform_diff_best,phase,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity
0,1,bdvegat,2,260.00,1,1.0,2.0,4.0,1.0,0.054187,0.052770
1,2,Phoenixest,2,185.83,1,0.0,1.0,3.0,1.0,0.017964,0.017805
2,3,HashNick,2,260.00,1,1.0,3.0,5.0,3.0,0.023697,0.023420
3,4,ccvacad,2,260.00,1,1.0,0.0,4.0,2.0,0.000000,0.000000
4,5,joaortizro,2,152.50,1,0.0,2.0,2.0,1.0,0.041420,0.040585
...,...,...,...,...,...,...,...,...,...,...,...
311,159,NicolasPrr,3,237.50,4,3.0,4.0,3.0,2.0,0.009360,0.009316
312,160,jumcorredorro,3,53.33,4,1.0,1.0,5.0,2.0,0.013294,0.013206
313,161,feartheGru,3,358.33,4,2.0,2.0,5.0,2.0,0.000000,0.000000
314,162,Danielsv9207,3,358.33,4,1.0,1.0,1.0,3.0,0.000000,0.000000


In [13]:
df_phase4 = df[df['phase'] == 4]
df_phase4

,Unnamed: 0,Student,group,abs_perform_diff_best,phase,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity
117,127,bdvegat,2,263.33,4,1.0,2.0,4.0,1.0,0.015866,0.015741
118,128,Phoenixest,2,263.33,4,0.0,1.0,3.0,1.0,0.016043,0.015915
119,129,HashNick,2,226.66,4,1.0,3.0,5.0,3.0,0.062130,0.060276
120,130,ccvacad,2,358.33,4,1.0,0.0,4.0,2.0,0.066406,0.064294
121,131,joaortizro,2,78.33,4,0.0,2.0,2.0,1.0,0.040816,0.040005
...,...,...,...,...,...,...,...,...,...,...,...
311,159,NicolasPrr,3,237.50,4,3.0,4.0,3.0,2.0,0.009360,0.009316
312,160,jumcorredorro,3,53.33,4,1.0,1.0,5.0,2.0,0.013294,0.013206
313,161,feartheGru,3,358.33,4,2.0,2.0,5.0,2.0,0.000000,0.000000
314,162,Danielsv9207,3,358.33,4,1.0,1.0,1.0,3.0,0.000000,0.000000


In [14]:
df_phase3 = df[df['phase'] == 3]
df_phase3

,Unnamed: 0,Student,group,abs_perform_diff_best,phase,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity
78,85,bdvegat,2,263.33,3,1.0,2.0,4.0,1.0,0.008547,0.008511
79,86,Phoenixest,2,263.33,3,0.0,1.0,3.0,1.0,0.036145,0.035507
80,87,HashNick,2,255.00,3,1.0,3.0,5.0,3.0,0.045082,0.044095
81,88,ccvacad,2,358.33,3,1.0,0.0,4.0,2.0,0.092199,0.088193
82,89,joaortizro,2,53.33,3,0.0,2.0,2.0,1.0,0.063830,0.061875
...,...,...,...,...,...,...,...,...,...,...,...
271,117,lizzyt10h,3,184.17,3,2.0,1.0,5.0,2.0,0.081633,0.078472
272,118,NicolasPrr,3,221.67,3,3.0,4.0,3.0,2.0,0.048847,0.047691
273,120,feartheGru,3,330.00,3,2.0,2.0,5.0,2.0,0.000000,0.000000
274,121,Danielsv9207,3,330.00,3,1.0,1.0,1.0,3.0,0.000000,0.000000


In [15]:
df_phase3 = pd.merge(df_phase3, commits[['Student', 'count', 'phase']], how='left', left_on=['Student', 'phase'], right_on =['Student', 'phase'] )
df_phase3['count'] = df_phase3['count'].fillna(0)
df_phase3 = df_phase3.drop(['Student', 'phase'], axis=1)
df_phase3 

,Unnamed: 0,group,abs_perform_diff_best,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity,count
0,85,2,263.33,1.0,2.0,4.0,1.0,0.008547,0.008511,2.0
1,86,2,263.33,0.0,1.0,3.0,1.0,0.036145,0.035507,0.0
2,87,2,255.00,1.0,3.0,5.0,3.0,0.045082,0.044095,0.0
3,88,2,358.33,1.0,0.0,4.0,2.0,0.092199,0.088193,0.0
4,89,2,53.33,0.0,2.0,2.0,1.0,0.063830,0.061875,1.0
...,...,...,...,...,...,...,...,...,...,...
74,117,3,184.17,2.0,1.0,5.0,2.0,0.081633,0.078472,3.0
75,118,3,221.67,3.0,4.0,3.0,2.0,0.048847,0.047691,1.0
76,120,3,330.00,2.0,2.0,5.0,2.0,0.000000,0.000000,0.0
77,121,3,330.00,1.0,1.0,1.0,3.0,0.000000,0.000000,0.0


In [16]:
df_phase4 = pd.merge(df_phase4, commits[['Student', 'count', 'phase']], how='left', left_on=['Student', 'phase'], right_on =['Student', 'phase'] )
df_phase4['count'] = df_phase4['count'].fillna(0)
df_phase4 = df_phase4.drop(['Student', 'phase'], axis=1)
df_phase4 

,Unnamed: 0,group,abs_perform_diff_best,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity,count
0,127,2,263.33,1.0,2.0,4.0,1.0,0.015866,0.015741,0.0
1,128,2,263.33,0.0,1.0,3.0,1.0,0.016043,0.015915,0.0
2,129,2,226.66,1.0,3.0,5.0,3.0,0.062130,0.060276,0.0
3,130,2,358.33,1.0,0.0,4.0,2.0,0.066406,0.064294,1.0
4,131,2,78.33,0.0,2.0,2.0,1.0,0.040816,0.040005,0.0
...,...,...,...,...,...,...,...,...,...,...
74,159,3,237.50,3.0,4.0,3.0,2.0,0.009360,0.009316,3.0
75,160,3,53.33,1.0,1.0,5.0,2.0,0.013294,0.013206,1.0
76,161,3,358.33,2.0,2.0,5.0,2.0,0.000000,0.000000,0.0
77,162,3,358.33,1.0,1.0,1.0,3.0,0.000000,0.000000,0.0


In [17]:
reg_expr = 'ln_similarity ~ group'
olsr_model = smf.ols(formula=reg_expr, data=df_phase4)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:          ln_similarity   R-squared:                       0.491
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     74.34
Date:                Fri, 25 Nov 2022   Prob (F-statistic):           6.42e-13
Time:                        16:15:23   Log-Likelihood:                 206.12
No. Observations:                  79   AIC:                            -408.2
Df Residuals:                      77   BIC:                            -403.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1119      0.010     10.790      0.0

In [18]:
reg_expr = 'ln_similarity ~ group'
olsr_model = smf.ols(formula=reg_expr, data=df_phase3)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:          ln_similarity   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.8857
Date:                Fri, 25 Nov 2022   Prob (F-statistic):              0.350
Time:                        16:15:24   Log-Likelihood:                 140.32
No. Observations:                  79   AIC:                            -276.6
Df Residuals:                      77   BIC:                            -271.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0376      0.024      1.578      0.1

In [19]:
df = pd.merge(df, commits[['Student', 'count', 'phase']], how='left', left_on=['Student', 'phase'], right_on =['Student', 'phase'] )
df['count'] = df['count'].fillna(0)
df = df.drop(['Student', 'phase'], axis=1)
df     

,Unnamed: 0,group,abs_perform_diff_best,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity,count
0,1,2,260.00,1.0,2.0,4.0,1.0,0.054187,0.052770,1.0
1,2,2,185.83,0.0,1.0,3.0,1.0,0.017964,0.017805,2.0
2,3,2,260.00,1.0,3.0,5.0,3.0,0.023697,0.023420,0.0
3,4,2,260.00,1.0,0.0,4.0,2.0,0.000000,0.000000,0.0
4,5,2,152.50,0.0,2.0,2.0,1.0,0.041420,0.040585,3.0
...,...,...,...,...,...,...,...,...,...,...
311,159,3,237.50,3.0,4.0,3.0,2.0,0.009360,0.009316,3.0
312,160,3,53.33,1.0,1.0,5.0,2.0,0.013294,0.013206,1.0
313,161,3,358.33,2.0,2.0,5.0,2.0,0.000000,0.000000,0.0
314,162,3,358.33,1.0,1.0,1.0,3.0,0.000000,0.000000,0.0


In [20]:
## Full Transparency case 

#Let's confirm that aspiration satisfy the relevance condition for performance distance to the best. There is a difference between aspiration and social aspiration. 
reg_expr = 'abs_perform_diff_best ~ Q10'

# Build and train an OLS model that regresses performance distance to the best on aspiration and verify
# using the F-test that coefficients of aspiration is significant 
olsr_model = smf.ols(formula=reg_expr, data=full)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

full['ln_similarity'] = np.log(full['similarity'] + 1)

# Build out the exog matrix. Statsmodels requires this matrix to contain all the endogenous and
# exogenous variables, plus the constant.
exog = full[['abs_perform_diff_best']]
exog = add_constant(exog)

# Build out the instruments matrix. Statsmodels requires this matrix to contain not only all the
# instruments but also the variables in exog that will NOT be instrumented
instruments = full[['Q10']]
instruments = add_constant(instruments)

#Build and train the IV2SLS model
iv2sls_model = IV2SLS(endog=full['ln_similarity'], exog=exog, instrument=instruments)
iv2sls_model_results = iv2sls_model.fit()

#Print the training summary
print(iv2sls_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_similarity ~ abs_perform_diff_best'
olsr_model = smf.ols(formula=reg_expr, data=full)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_similarity ~ Q10'
olsr_model = smf.ols(formula=reg_expr, data=full)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

                              OLS Regression Results                             
Dep. Variable:     abs_perform_diff_best   R-squared:                       0.086
Model:                               OLS   Adj. R-squared:                  0.080
Method:                    Least Squares   F-statistic:                     14.87
Date:                   Fri, 25 Nov 2022   Prob (F-statistic):           0.000167
Time:                           16:15:25   Log-Likelihood:                -913.78
No. Observations:                    160   AIC:                             1832.
Df Residuals:                        158   BIC:                             1838.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    258.9941 

In [21]:
## Full Transparency case 

#Let's confirm that aspiration satisfy the relevance condition for performance distance to the best. There is a difference between aspiration and social aspiration. 
reg_expr = 'abs_perform_diff_best ~ Q10'

# Build and train an OLS model that regresses performance distance to the best on aspiration and verify
# using the F-test that coefficients of aspiration is significant 
olsr_model = smf.ols(formula=reg_expr, data=solution)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

solution['ln_similarity'] = np.log(solution['similarity'] + 1)

# Build out the exog matrix. Statsmodels requires this matrix to contain all the endogenous and
# exogenous variables, plus the constant.
exog = solution[['abs_perform_diff_best']]
exog = add_constant(exog)

# Build out the instruments matrix. Statsmodels requires this matrix to contain not only all the
# instruments but also the variables in exog that will NOT be instrumented
instruments = solution[['Q10']]
instruments = add_constant(instruments)

#Build and train the IV2SLS model
iv2sls_model = IV2SLS(endog=solution['ln_similarity'], exog=exog, instrument=instruments)
iv2sls_model_results = iv2sls_model.fit()

#Print the training summary
print(iv2sls_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_similarity ~ abs_perform_diff_best'
olsr_model = smf.ols(formula=reg_expr, data=solution)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())


                              OLS Regression Results                             
Dep. Variable:     abs_perform_diff_best   R-squared:                       0.002
Model:                               OLS   Adj. R-squared:                 -0.005
Method:                    Least Squares   F-statistic:                    0.2540
Date:                   Fri, 25 Nov 2022   Prob (F-statistic):              0.615
Time:                           16:15:26   Log-Likelihood:                -915.75
No. Observations:                    156   AIC:                             1836.
Df Residuals:                        154   BIC:                             1842.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    199.5418 

In [22]:
#Let's confirm that aspiration satisfy the relevance condition for performance distance to the best. There is a difference between aspiration and social aspiration. 
reg_expr = 'abs_perform_diff_best ~ Q10'

# Build and train an OLS model that regresses performance distance to the best on aspiration and verify
# using the F-test that coefficients of aspiration is significant 
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

df['ln_similarity'] = np.log(df['similarity'] + 1)
df['ln_perform_diff'] = np.log(df['abs_perform_diff_best']+1)
df['ln_count'] = np.log(df['count']+1)

# Build out the exog matrix. Statsmodels requires this matrix to contain all the endogenous and
# exogenous variables, plus the constant.
exog = df[['abs_perform_diff_best']]
exog = add_constant(exog)

# Build out the instruments matrix. Statsmodels requires this matrix to contain not only all the
# instruments but also the variables in exog that will NOT be instrumented
instruments = df[['group']]
instruments = add_constant(instruments)

#Build and train the IV2SLS model
iv2sls_model = IV2SLS(endog=df['ln_similarity'], exog=exog, instrument=instruments)
iv2sls_model_results = iv2sls_model.fit()

#Print the training summary
print(iv2sls_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_similarity ~ group + abs_perform_diff_best + Q7_Q7_1 + Q7_Q7_2 + Q8_Q8_1 + Q10 +  group * abs_perform_diff_best'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_similarity ~ group + abs_perform_diff_best + Q7_Q7_1 + Q7_Q7_2 + Q8_Q8_1 + Q10'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_similarity ~ group'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_perform_diff ~ group'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_similarity ~ group + abs_perform_diff_best + group * abs_perform_diff_best'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_similarity ~ group + abs_perform_diff_best + Q10 + group * abs_perform_diff_best'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_similarity ~ group + abs_perform_diff_best + Q10'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

reg_expr = 'ln_count ~ group'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

                              OLS Regression Results                             
Dep. Variable:     abs_perform_diff_best   R-squared:                       0.023
Model:                               OLS   Adj. R-squared:                  0.020
Method:                    Least Squares   F-statistic:                     7.361
Date:                   Fri, 25 Nov 2022   Prob (F-statistic):            0.00703
Time:                           16:15:27   Log-Likelihood:                -1834.9
No. Observations:                    316   AIC:                             3674.
Df Residuals:                        314   BIC:                             3681.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    226.1419 

## Novelty

In [23]:
#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best

novelty['ln_novelty'] = np.log(novelty['novelty'] + 1)
reg_expr = 'ln_novelty ~ group'
olsr_model = smf.ols(formula=reg_expr, data=novelty)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:             ln_novelty   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     7.625
Date:                Fri, 25 Nov 2022   Prob (F-statistic):            0.00608
Time:                        16:15:28   Log-Likelihood:                 30.781
No. Observations:                 332   AIC:                            -57.56
Df Residuals:                     330   BIC:                            -49.95
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2790      0.062      4.517      0.0

In [24]:
pd.set_option('display.max_rows', None)
novelty

,Student,group,abs_perform_diff_best,phase,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,user2,novelty,ln_novelty
0,lemartinp,3,270.84,1,0.0,0.0,4.0,1.0,0.146341,lemartinp,0.708333,0.535518
1,juligarji,3,167.50,1,4.0,4.0,5.0,3.0,0.057325,juligarji,0.942675,0.664066
2,nfmorenog,3,225.00,1,0.0,2.0,3.0,1.0,0.190476,nfmorenog,0.745098,0.556811
3,AFelipeGA,3,291.67,1,4.0,3.0,5.0,2.0,0.090909,AFelipeGA,0.400000,0.336472
4,mballeng91,3,329.17,1,2.0,6.0,6.0,2.0,0.004854,mballeng91,0.968912,0.677481
5,dduartec,3,287.50,1,1.0,5.0,4.0,1.0,0.092308,dduartec,0.666667,0.510826
6,srmedinac,3,304.17,1,1.0,2.0,4.0,1.0,0.059524,srmedinac,0.878378,0.630409
7,wilson911013,3,250.84,1,0.0,1.0,2.0,1.0,0.000000,wilson911013,0.000000,0.000000
8,wapiravaguens,3,90.00,1,0.0,1.0,4.0,2.0,0.090909,wapiravaguens,0.400000,0.336472
9,davidh17,3,173.34,1,3.0,3.0,5.0,3.0,0.119403,davidh17,0.616438,0.480225


In [ ]:
novelty_phase4 = novelty[novelty['phase']==4]
novelty_phase4

In [ ]:
novelty_phase4_group3 = novelty_phase4[novelty_phase4['group'] == 3]
novelty_phase4_group3['ln_novelty'].mean()

In [ ]:
novelty_phase4_group2 = novelty_phase4[novelty_phase4['group'] == 2]
novelty_phase4_group2

In [ ]:
g3 = df[df['group']==3]
g3['ln_similarity'].mean()

In [ ]:
g2 = df[df['group']==2]
g2['ln_similarity'].mean()

> Comment: Higher similarity to the best in full transparency group 

In [ ]:
reg_expr = 'ln_novelty ~ group + abs_perform_diff_best + Q7_Q7_1 + Q7_Q7_2 + Q8_Q8_1 + Q10 + group * abs_perform_diff_best'
olsr_model = smf.ols(formula=reg_expr, data=novelty)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

In [ ]:
reg_expr = 'ln_novelty ~ group + abs_perform_diff_best + Q7_Q7_1 + Q7_Q7_2 + Q8_Q8_1 + Q10'
olsr_model = smf.ols(formula=reg_expr, data=novelty)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

In [ ]:
reg_expr = 'ln_novelty ~ abs_perform_diff_best + Q7_Q7_1 + Q7_Q7_2 + Q8_Q8_1 + Q10'
olsr_model = smf.ols(formula=reg_expr, data=novelty)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

In [ ]:
g2 = novelty[novelty['group']==2]
g2['ln_novelty'].mean()

In [ ]:
g3 = novelty[novelty['group']==3]
g3['ln_novelty'].mean()

> Comment: Higher relational novelty in full transparency group

In [ ]:
#Let's confirm that aspiration satisfy the relevance condition for performance distance to the best. There is a difference between aspiration and social aspiration. 
novelty['ln_novelty'] = np.log(novelty['novelty'] + 1)

reg_expr = 'ln_novelty ~ similarity + group + group * similarity'

# Build and train an OLS model that regresses performance distance to the best on aspiration and verify
# using the F-test that coefficients of aspiration is significant 
olsr_model = smf.ols(formula=reg_expr, data=novelty)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

In [ ]:
#Let's confirm that aspiration satisfy the relevance condition for performance distance to the best. There is a difference between aspiration and social aspiration. 
g3['ln_novelty'] = np.log(g3['novelty'] + 1)

reg_expr = 'ln_novelty ~ similarity'

# Build and train an OLS model that regresses performance distance to the best on aspiration and verify
# using the F-test that coefficients of aspiration is significant 
olsr_model = smf.ols(formula=reg_expr, data=g3)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

In [ ]:
#Let's confirm that aspiration satisfy the relevance condition for performance distance to the best. There is a difference between aspiration and social aspiration. 
g2['ln_novelty'] = np.log(g2['novelty'] + 1)

reg_expr = 'ln_novelty ~ similarity'

# Build and train an OLS model that regresses performance distance to the best on aspiration and verify
# using the F-test that coefficients of aspiration is significant 
olsr_model = smf.ols(formula=reg_expr, data=g2)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

In [ ]:
#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best

overall_p1 = overall_23[['Group','total-phase1']]
overall_p2 = overall_23[['Group','total-phase2']]
overall_p3 = overall_23[['Group','total-phase3']]
overall_p4 = overall_23[['Group','total-phase4']]
overall_p5 = overall_23[['Group','total-phase5']]

overall_p1 = overall_p1.rename(columns={'total-phase1': 'total'})
overall_p2 = overall_p2.rename(columns={'total-phase2': 'total'})
overall_p3 = overall_p3.rename(columns={'total-phase3': 'total'})
overall_p4 = overall_p4.rename(columns={'total-phase4': 'total'})
overall_p5 = overall_p5.rename(columns={'total-phase5': 'total'})

total_sum = pd.concat([ overall_p1, overall_p2, overall_p3, overall_p4, overall_p5], ignore_index=True)

total_sum['ln_total'] = np.log(total_sum['total'] + 1)
reg_expr = 'ln_total ~ Group'
olsr_model = smf.ols(formula=reg_expr, data=total_sum)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())


In [ ]:
g3 = total_sum[total_sum['Group']==3]
g3['ln_total'].mean()

In [ ]:
g2 = total_sum[total_sum['Group']==2]
g2['ln_total'].mean()

> Full Transparency has a significantly higher total score compare to solution transparency

In [ ]:
#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best

overall_p1 = overall_23[['Group','novelty-phase1']]
overall_p2 = overall_23[['Group','novelty-phase2']]
overall_p3 = overall_23[['Group','novelty-phase3']]
overall_p4 = overall_23[['Group','novelty-phase4']]
overall_p5 = overall_23[['Group','novelty-phase5']]

overall_p1 = overall_p1.rename(columns={'novelty-phase1': 'novelty'})
overall_p2 = overall_p2.rename(columns={'novelty-phase2': 'novelty'})
overall_p3 = overall_p3.rename(columns={'novelty-phase3': 'novelty'})
overall_p4 = overall_p4.rename(columns={'novelty-phase4': 'novelty'})
overall_p5 = overall_p5.rename(columns={'novelty-phase5': 'novelty'})

novelty_sum = pd.concat([overall_p1, overall_p2, overall_p3, overall_p4, overall_p5], ignore_index=True)

novelty_sum['ln_novelty'] = np.log(novelty_sum['novelty'] + 1)
reg_expr = 'ln_novelty ~ Group'
olsr_model = smf.ols(formula=reg_expr, data=novelty_sum)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

In [ ]:
overall_p1 = overall_23[['Group', 'Student', 'user-requirement-phase1']]
overall_p2 = overall_23[['Group', 'Student', 'user-requirement-phase2']]
overall_p3 = overall_23[['Group', 'Student', 'user-requirement-phase3']]
overall_p4 = overall_23[['Group', 'Student', 'user-requirement-phase4']]
overall_p5 = overall_23[['Group', 'Student', 'user-requirement-phase5']]


overall_p1 = overall_p1.rename(columns={'user-requirement-phase1': 'user-requirement'})
overall_p2 = overall_p2.rename(columns={'user-requirement-phase2': 'user-requirement'})
overall_p3 = overall_p3.rename(columns={'user-requirement-phase3': 'user-requirement'})
overall_p4 = overall_p4.rename(columns={'user-requirement-phase4': 'user-requirement'})
overall_p5 = overall_p5.rename(columns={'user-requirement-phase5': 'user-requirement'})

overall_p1['phase'] = 1
overall_p2['phase'] = 2
overall_p3['phase'] = 3
overall_p4['phase'] = 4
overall_p5['phase'] = 5

user_requirement_sum = pd.concat([overall_p2, overall_p3, overall_p4, overall_p5], ignore_index=True)
confound = solution.append(full, ignore_index=True)
confound = confound[['Student', 'group', 'Q7_Q7_1', 'Q7_Q7_2', 'Q8_Q8_1', 'Q10', 'similarity']]
confound = confound.drop_duplicates(subset=['Student']) 
df = pd.merge(user_requirement_sum[['Student', 'Group', 'phase', 'user-requirement']], confound, how='left', left_on=['Student', 'Group'], right_on =['Student', 'group']) 

df['ln_user_requirement'] = np.log(df['user-requirement'] + 1)
reg_expr = 'ln_user_requirement ~ Group'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

df['ln_similarity'] = np.log(df['similarity'] + 1)
reg_expr = 'ln_user_requirement ~ Group'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

In [ ]:
overall_p1 = overall_23[['Group', 'Student', 'user-requirement-phase1']]
overall_p2 = overall_23[['Group', 'Student', 'user-requirement-phase2']]
overall_p3 = overall_23[['Group', 'Student', 'user-requirement-phase3']]
overall_p4 = overall_23[['Group', 'Student', 'user-requirement-phase4']]
overall_p5 = overall_23[['Group', 'Student', 'user-requirement-phase5']]


overall_p1 = overall_p1.rename(columns={'user-requirement-phase1': 'user-requirement'})
overall_p2 = overall_p2.rename(columns={'user-requirement-phase2': 'user-requirement'})
overall_p3 = overall_p3.rename(columns={'user-requirement-phase3': 'user-requirement'})
overall_p4 = overall_p4.rename(columns={'user-requirement-phase4': 'user-requirement'})
overall_p5 = overall_p5.rename(columns={'user-requirement-phase5': 'user-requirement'})

overall_p1['phase'] = 1
overall_p2['phase'] = 2
overall_p3['phase'] = 3
overall_p4['phase'] = 4
overall_p5['phase'] = 5

user_requirement_sum = pd.concat([overall_p2, overall_p3, overall_p4, overall_p5], ignore_index=True)
novelty = novelty[['Student', 'group', 'Q7_Q7_1', 'Q7_Q7_2', 'Q8_Q8_1', 'Q10', 'similarity', 'novelty']]
novelty = novelty.drop_duplicates(subset=['Student']) 
df = pd.merge(user_requirement_sum[['Student', 'Group', 'phase', 'user-requirement']], novelty, how='left', left_on=['Student', 'Group'], right_on =['Student', 'group']) 

df['ln_user_requirement'] = np.log(df['user-requirement'] + 1)
reg_expr = 'ln_user_requirement ~ Group'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

df['ln_similarity'] = np.log(df['similarity'] + 1)
df['ln_novelty'] = np.log(df['novelty'] + 1)
df['user_requirement'] = df['user-requirement']
reg_expr = 'ln_user_requirement ~ novelty'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

df['ln_similarity'] = np.log(df['similarity'] + 1)
df['ln_novelty'] = np.log(df['novelty'] + 1)
df['user_requirement'] = df['user-requirement']
reg_expr = 'ln_user_requirement ~ similarity'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())


In [ ]:
g3 = df[df['Group']==3]
g3['ln_user_requirement'].mean()

In [ ]:
g2 = df[df['Group']==2]
g2['ln_user_requirement'].mean()

In [ ]:
# Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best

overall_p1 = overall_23[['Group', 'Student', 'infovis-phase1']]
overall_p2 = overall_23[['Group', 'Student', 'infovis-phase2']]
overall_p3 = overall_23[['Group', 'Student', 'infovis-phase3']]
overall_p4 = overall_23[['Group', 'Student', 'infovis-phase4']]
overall_p5 = overall_23[['Group', 'Student', 'infovis-phase5']]

overall_p1 = overall_p1.rename(columns={'infovis-phase1': 'infovis'})
overall_p2 = overall_p2.rename(columns={'infovis-phase2': 'infovis'})
overall_p3 = overall_p3.rename(columns={'infovis-phase3': 'infovis'})
overall_p4 = overall_p4.rename(columns={'infovis-phase4': 'infovis'})
overall_p5 = overall_p5.rename(columns={'infovis-phase5': 'infovis'})

infovis_sum = pd.concat([overall_p1, overall_p2, overall_p3, overall_p4, overall_p5], ignore_index=True)

infovis_sum['ln_infovis'] = np.log(infovis_sum['infovis'] + 1)
reg_expr = 'ln_infovis ~ Group'
olsr_model = smf.ols(formula=reg_expr, data=infovis_sum)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

In [ ]:
# Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best

overall_p1 = overall_23[['Group','tech-phase1']]
overall_p2 = overall_23[['Group','tech-phase2']]
overall_p3 = overall_23[['Group','tech-phase3']]
overall_p4 = overall_23[['Group','tech-phase4']]
overall_p5 = overall_23[['Group','tech-phase5']]

overall_p1 = overall_p1.rename(columns={'tech-phase1': 'tech'})
overall_p2 = overall_p2.rename(columns={'tech-phase2': 'tech'})
overall_p3 = overall_p3.rename(columns={'tech-phase3': 'tech'})
overall_p4 = overall_p4.rename(columns={'tech-phase4': 'tech'})
overall_p5 = overall_p5.rename(columns={'tech-phase5': 'tech'})

tech_sum = pd.concat([overall_p1, overall_p2, overall_p3, overall_p4, overall_p5], ignore_index=True)

tech_sum['ln_tech'] = np.log(tech_sum['tech'] + 1)
reg_expr = 'ln_tech ~ Group'
olsr_model = smf.ols(formula=reg_expr, data=tech_sum)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

In [ ]:
df

In [ ]:
%store novelty